# Exploratory data analysis (EDA)

Use this notebook to explore and visualize the data.

In [1]:
import os
import ibis
import ibis.selectors as s

from icarus.config import (
    DATA_DIR,
    RAW_DATA_DIR,
    RAW_BUY_SELL_TABLE,
    RAW_SOCIAL_MEDIA_TABLE,
    BRONZE,
    SILVER,
    GOLD,
)
from icarus.synthetic_data import fake, seed_table
from icarus.investments.dag.resources import Catalog

ibis.options.interactive = True

In [18]:
f"{10_000_000_000 / (60 * 60 * 24):,.2f}"

'115,740.74'

In [2]:
lookback = ibis.interval(days=1)
step = ibis.interval(seconds=1)

In [11]:
t = (
    ibis.range(ibis.now() - lookback, ibis.now(), step=step)
    .unnest()
    .name("timestamp")
    .as_table()
    .mutate(
        index=(ibis.row_number().over(order_by="timestamp")),
        symbol=ibis.random(),
        bid=ibis.random(),
        bid_size=ibis.random(),
        ask=ibis.random(),
        ask_size=ibis.random(),
        **{c: 2 * (ibis.random() - 0.5) for c in ["a", "b", "c"]},
        **{
            c: ibis.null(type=str)
            for c in list(map(chr, range(ord("d"), ord("z") + 1)))
        },
    )
    .relocate("index", "timestamp")
)
t

┏━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┓
┃ index ┃ timestamp               ┃ symbol   ┃ bid      ┃ bid_size ┃ ask      ┃ ask_size ┃ a         ┃ b         ┃ c         ┃ d      ┃ e      ┃ f      ┃ g      ┃ h      ┃ i      ┃ j      ┃ k      ┃ l      ┃ m      ┃ n      ┃ o      ┃ p      ┃ q      ┃ r      ┃ s      ┃ t      ┃ u      ┃ v      ┃ w      ┃ x      ┃ y      ┃ z      ┃
┡━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━┩
│ int64 │ timestamp               │ float64  │ float64  │ float64  │ float64  │ float64  │ float64   │ float64   │ float64   │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │
├───────┼─────────────────────────┼──────────┼──────────┼──────────┼──────────┼──────────┼───────────┼───────────┼───────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┤
│     0 │ 2024-08-11 22:19:11.575 │ 0.537716 │ 0.178323 │ 0.426736 │ 0.115570 │ 0.601905 │ -0.983770 │  0.403018 │  0.279080 │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │
│     1 │ 2024-08-11 22:19:12.575 │ 0.398241 │ 0.289968 │ 0.180243 │ 0.019478 │ 0.709004 │  0.898992 │  0.987540 │ -0.357706 │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │
│     2 │ 2024-08-11 22:19:13.575 │ 0.010442 │ 0.111750 │ 0.460888 │ 0.524075 │ 0.264093 │ -0.852427 │ -0.679882 │ -0.906654 │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │
│     3 │ 2024-08-11 22:19:14.575 │ 0.854956 │ 0.996160 │ 0.734763 │ 0.060142 │ 0.509190 │  0.012359 │  0.689564 │ -0.410368 │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │
│     4 │ 2024-08-11 22:19:15.575 │ 0.139906 │ 0.712218 │ 0.676738 │ 0.643007 │ 0.385706 │ -0.378164 │  0.868036 │ -0.719976 │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │
│     5 │ 2024-08-11 22:19:16.575 │ 0.995277 │ 0.060174 │ 0.307790 │ 0.350550 │ 0.931032 │ -0.698325 │  0.064596 │  0.381836 │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │
│     6 │ 2024-08-11 22:19:17.575 │ 0.537642 │ 0.547164 │ 0.741763 │ 0.709485 │ 0.316319 │ -0.789704 │  0.397333 │  0.549758 │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ N

In [12]:
import random

symbols = [chr(i) for i in range(ord("A"), ord("Z") + 1)]
symbols = [
    f"{a}{b}{c}{d}" for a in symbols for b in symbols for c in symbols for d in symbols
]
symbols = random.sample(symbols, 500)
symbols[:5]

['ZRZH', 'RMCH', 'PNHV', 'ABLJ', 'GMZK']

In [13]:
symbols_case = ibis.case()
for i, symbol in enumerate(symbols):
    symbols_case = symbols_case.when(
        (i / len(symbols) < ibis._["symbol"])
        & (ibis._["symbol"] < ((i + 1) / len(symbols))),
        symbol,
    )
symbols_case = symbols_case.else_(ibis.null()).end()

In [14]:
t = t.mutate(
    symbol=symbols_case,
    bid_size=(ibis._["bid_size"] * 1000).cast("int64"),
    ask_size=(ibis._["ask_size"] * 1000).cast("int64"),
)
t

┏━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┓
┃ index ┃ timestamp               ┃ symbol ┃ bid      ┃ bid_size ┃ ask      ┃ ask_size ┃ a         ┃ b         ┃ c         ┃ d      ┃ e      ┃ f      ┃ g      ┃ h      ┃ i      ┃ j      ┃ k      ┃ l      ┃ m      ┃ n      ┃ o      ┃ p      ┃ q      ┃ r      ┃ s      ┃ t      ┃ u      ┃ v      ┃ w      ┃ x      ┃ y      ┃ z      ┃
┡━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━┩
│ int64 │ timestamp               │ string │ float64  │ int64    │ float64  │ int64    │ float64   │ float64   │ float64   │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │
├───────┼─────────────────────────┼────────┼──────────┼──────────┼──────────┼──────────┼───────────┼───────────┼───────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┤
│     0 │ 2024-08-11 22:19:12.748 │ VDWO   │ 0.823168 │      289 │ 0.991180 │      269 │  0.301875 │ -0.662264 │ -0.284875 │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │
│     1 │ 2024-08-11 22:19:13.748 │ VGYC   │ 0.364646 │      471 │ 0.241433 │      562 │ -0.905707 │  0.370740 │  0.289428 │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │
│     2 │ 2024-08-11 22:19:14.748 │ VMZP   │ 0.829490 │      190 │ 0.070859 │      596 │  0.785806 │ -0.185280 │ -0.049335 │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │
│     3 │ 2024-08-11 22:19:15.748 │ BTPB   │ 0.266551 │      840 │ 0.546260 │      456 │  0.631800 │ -0.586122 │  0.699715 │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │
│     4 │ 2024-08-11 22:19:16.748 │ XFSU   │ 0.320441 │      998 │ 0.258961 │      532 │  0.088285 │  0.566166 │ -0.255831 │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │
│     5 │ 2024-08-11 22:19:17.748 │ SGNM   │ 0.615721 │      431 │ 0.773898 │      150 │ -0.816429 │  0.083019 │  0.380042 │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │
│     6 │ 2024-08-11 22:19:18.748 │ EXGK   │ 0.251257 │      101 │ 0.005470 │       83 │  0.649196 │ -0.729369 │ -0.174144 │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │
│     7 │ 2024-0

In [15]:
t

┏━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┓
┃ index ┃ timestamp               ┃ symbol ┃ bid      ┃ bid_size ┃ ask      ┃ ask_size ┃ a         ┃ b         ┃ c         ┃ d      ┃ e      ┃ f      ┃ g      ┃ h      ┃ i      ┃ j      ┃ k      ┃ l      ┃ m      ┃ n      ┃ o      ┃ p      ┃ q      ┃ r      ┃ s      ┃ t      ┃ u      ┃ v      ┃ w      ┃ x      ┃ y      ┃ z      ┃
┡━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━┩
│ int64 │ timestamp               │ string │ float64  │ int64    │ float64  │ int64    │ float64   │ float64   │ float64   │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │ string │
├───────┼─────────────────────────┼────────┼──────────┼──────────┼──────────┼──────────┼───────────┼───────────┼───────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┼────────┤
│     0 │ 2024-08-11 22:19:13.616 │ OJGP   │ 0.418952 │       49 │ 0.059946 │      732 │ -0.859806 │ -0.073967 │  0.100235 │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │
│     1 │ 2024-08-11 22:19:14.616 │ ISWK   │ 0.785870 │       65 │ 0.283215 │       86 │ -0.287481 │ -0.069651 │  0.364331 │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │
│     2 │ 2024-08-11 22:19:15.616 │ NKUP   │ 0.065686 │      394 │ 0.281836 │       77 │  0.905348 │  0.214216 │ -0.281444 │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │
│     3 │ 2024-08-11 22:19:16.616 │ BJPS   │ 0.704723 │      981 │ 0.917297 │      929 │ -0.092330 │  0.989667 │ -0.530136 │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │
│     4 │ 2024-08-11 22:19:17.616 │ YAFI   │ 0.086378 │      898 │ 0.080285 │      229 │ -0.514993 │ -0.240314 │ -0.199076 │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │
│     5 │ 2024-08-11 22:19:18.616 │ ABSN   │ 0.636502 │      740 │ 0.092708 │      365 │  0.169621 │ -0.994266 │  0.159818 │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │
│     6 │ 2024-08-11 22:19:19.616 │ IRAI   │ 0.059001 │      549 │ 0.761799 │      265 │  0.988877 │  0.292978 │ -0.882335 │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │ NULL   │
│     7 │ 2024-0

In [ ]:
seed_table

In [ ]:
catalog = Catalog()
catalog.list_groups()

In [ ]:
catalog.list_tables(GOLD)

In [ ]:
seed_table

In [ ]:
data_glob = os.path.join(DATA_DIR, RAW_DATA_DIR, RAW_BUY_SELL_TABLE, "*.parquet")
bs = ibis.read_parquet(data_glob)
bs

In [ ]:
bs = catalog.table("bronze_buy_sell")
bs

In [ ]:
bs = catalog.table("silver_buy_sell")
bs

In [ ]:
bs = catalog.table("gold_buy_sell")
bs

In [ ]:
bs.count()

In [ ]:
bs["location"].value_counts()

In [ ]:
f"{bs.count().to_pyarrow().as_py():,}"

In [ ]:
data_glob = os.path.join(DATA_DIR, RAW_DATA_DIR, RAW_SOCIAL_MEDIA_TABLE, "*.parquet")
sm = ibis.read_parquet(data_glob)
sm

In [ ]:
sm = catalog.table("silver_social_media")
sm